# Steam project

## Importing data

In [0]:
# Import useful libraries and the dataset
from pyspark.sql import functions as F # This will load the class where spark sql functions are contained
from pyspark.sql import Row # this will let us manipulate rows with spark sql
from pyspark.sql.types import * # Import types to convert columns using spark sql
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY") # Facilitate date formatting

filename = 's3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json'
data = spark.read.format('json').load(filename)

It seems that the dataset has a nested schema, it is important to understand the structure before any analysis.

In [0]:
data.describe().toPandas()

,summary,id
0,count,55691
1,mean,1025603.0926720655
2,stddev,522784.96832834504
3,min,10
4,max,999990


In [0]:
# Display the first lines
data.show(5)

# Display the structure
data.printSchema()

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
+--------------------+-------+
only showing top 5 rows

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolea

There is 2 columns in the dataframe but the schema is much more complicated because of nested structures.
The next step is investigating the structure and flatten the dataframe in order to prepare the exploratory data analysis.

## Data cleaning and data selection

In [0]:
# First flattening of the dataframe
df = data.select("data.*","id")
df.limit(5).toPandas()

,appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,...,positive,price,publisher,release_date,required_age,short_description,tags,type,website,id
0,10,"[Multi-player, Valve Anti-Cheat enabled, Onlin...",13990,Valve,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, French, German, Italian, Spanish - Sp...",Counter-Strike,...,201215,999,Valve,2000/11/1,0,Play the world's number 1 online action game. ...,"{'1980s': 266.0, '1990's': 1191.0, '2.5D': Non...",game,,10
1,1000000,"[Single-player, Partial Controller Support, St...",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, Korean, Simplified Chinese",ASCENXION,...,27,999,PsychoFlux Entertainment,2021/05/14,0,ASCENXION is a 2D shoot 'em up game where you ...,"{'1980s': None, '1990's': None, '2.5D': None, ...",game,,1000000
2,1000010,"[Single-player, Partial Controller Support, St...",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",https://cdn.akamai.steamstatic.com/steam/apps/...,1999,"Simplified Chinese, English, Japanese, Traditi...",Crown Trick,...,4032,599,"Team17, NEXT Studios",2020/10/16,0,"Enter a labyrinth that moves as you move, wher...","{'1980s': None, '1990's': None, '2.5D': None, ...",game,,1000010
3,1000030,"[Multi-player, Single-player, Co-op, Steam Ach...",76,Vertigo Gaming Inc.,0,"Action, Indie, Simulation, Strategy",https://cdn.akamai.steamstatic.com/steam/apps/...,1999,English,"Cook, Serve, Delicious! 3?!",...,1575,1999,Vertigo Gaming Inc.,2020/10/14,0,"Cook, serve and manage your food truck as you ...","{'1980s': None, '1990's': None, '2.5D': None, ...",game,http://www.cookservedelicious.com,1000030
4,1000040,[Single-player],0,DoubleC Games,0,"Action, Casual, Indie, Simulation",https://cdn.akamai.steamstatic.com/steam/apps/...,199,Simplified Chinese,细胞战争,...,0,199,DoubleC Games,2019/03/30,0,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通...,"{'1980s': None, '1990's': None, '2.5D': None, ...",game,,1000040


In [0]:
df.printSchema()

root
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms: struct (nullable = true)
 |    |-- linux: boolean (nullable = true)
 |    |-- mac: boolean (nullable = true)
 |    |-- windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- tags: struct (nullable = true)
 |    |-- 1980s: lon

In [0]:
# Modify types for some columns
df = df \
    .withColumn('id', F.col("id").cast(IntegerType())) \
    .withColumn('discount', F.col("discount").cast(IntegerType())) \
    .withColumn('initialprice', F.col("initialprice").cast(IntegerType())) \
    .withColumn('price', F.col("price").cast(IntegerType())) \
    .withColumn("release_date", F.to_date(df["release_date"], "yyyy/MM/dd"))\
    .withColumn("release_year", F.year(F.col('release_date'))) \
    .withColumn('required_age', F.col("required_age").cast(IntegerType()))

# Drop columns and filter only necessary data
df = df \
    .filter(F.col('type') == "game") \
    .withColumnRenamed('ccu_count', 'ccu') \
    .drop('header_image', 'website', 'type', 'short_description')

# Ensure both column are identical before dropping
if df.filter(F.col('appid') != F.col('id')).count() == 0 :
    df = df.drop('id')
else:
    pass

In [0]:
# Duplicates check
df.count() - df.dropDuplicates().count()

Out[41]: 0

In [0]:
# Create a function to check any outliers before visualization
def check_values(column):
    df \
        .select(column) \
        .distinct() \
        .sort(column, ascending=False) \
        .show(3)
    
    df \
        .select(column) \
        .distinct() \
        .sort(column, ascending=True) \
        .show(3)

## Data analysis - macro level

##### Which publisher has released the most games on Steam?

In [0]:
#Top 15 publishers on Steam
df_dev = df \
    .filter(F.col("publisher") != "") \
    .groupBy('publisher') \
    .count() \
    .sort('count', ascending=False) \
    .limit(15)

df_dev = df_dev.na.drop()

display(df_dev)

publisher,count
Big Fish Games,422
8floor,202
SEGA,165
Strategy First,151
Square Enix,141
Choice of Games,140
Sekai Project,132
HH-Games,132
Ubisoft,127
Laush Studio,126


Databricks visualization. Run in Databricks to view.

##### What are the best rated games ?

In [0]:
#Top 15 best-rated games on Steam
df_evaluation_pos = df \
    .sort("positive", ascending=False) \
    .limit(15)

display(df_evaluation_pos)

appid,categories,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,tags,release_year
730,"List(Multi-player, Valve Anti-Cheat enabled, Stats, Steam Achievements, Full controller support, Steam Trading Cards, Steam Workshop, In-App Purchases, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV)",874053,"Valve, Hidden Path Entertainment",0,"Action, Free to Play",0,"English, Czech, Danish, Dutch, Finnish, French, German, Hungarian, Italian, Japanese, Korean, Norwegian, Polish, Portuguese - Portugal, Portuguese - Brazil, Romanian, Russian, Simplified Chinese, Spanish - Spain, Swedish, Thai, Traditional Chinese, Turkish, Bulgarian, Ukrainian, Greek, Spanish - Latin America, Vietnamese",Counter-Strike: Global Offensive,787093,"50,000,000 .. 100,000,000","List(true, true, true)",5943345,0,Valve,2012-08-21,0,"List(null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 46305, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 29499, null, null, null, null, null, null, null, null, null, null, 51794, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 25454, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 88154, null, null, null, null, null, 24811, null, null, null, 38310, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 28011, null, null, null, null, null, 6107, null, null, null, null, null, 60476, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 33089, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 33419, null, null, null, null, null, null, null, null, 24863, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 63364, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 29366, null, null, null, null, null, null, null, null, null, null, 40298, null, null, 45204, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 25026, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 27408, null, null, null, null, null, null, null, null, null, null, null, null, 42184)",2012
570,"List(Multi-player, Valve Anti-Cheat enabled, Co-op, Steam Trading Cards, Steam Workshop, In-App Purchases, SteamVR Collectibles)",852995,Valve,0,"Action, Free to Play, Strategy",0,"English, Bulgarian, Czech, Danish, Dutch, Finnish, French, German, Greek, Hungarian, Italian, Japanese, Korean, Norwegian, Polish, Portuguese - Portugal, Portuguese - Brazil, Romanian, Russian, Simplified Chinese, Spanish - Spain,

Databricks visualization. Run in Databricks to view.

In [0]:
#Top 15 worst-rated games on Steam
df_evaluation_neg = df \
    .sort("negative", ascending=False) \
    .limit(15)

display(df_evaluation_neg)

appid,categories,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,tags,release_year
578080,"List(Multi-player, Stats, Online PvP, Remote Play on Phone, Remote Play on Tablet, PvP)",339287,"KRAFTON, Inc.",0,"Action, Adventure, Free to Play, Massively Multiplayer",0,"English, Korean, Simplified Chinese, French, German, Spanish - Spain, Arabic, Japanese, Polish, Portuguese - Portugal, Russian, Turkish, Thai, Italian, Portuguese - Brazil, Traditional Chinese, Ukrainian",PUBG: BATTLEGROUNDS,908515,"50,000,000 .. 100,000,000","List(false, false, true)",1185361,0,"KRAFTON, Inc.",2017-12-21,0,"List(null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 5728, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 10218, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 4074, null, null, null, null, null, null, null, null, null, null, 2208, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1596, null, null, null, null, null, null, null, null, null, null, null, 2737, null, null, null, null, null, null, null, null, null, null, null, null, 7852, null, null, null, null, null, null, null, null, null, 3828, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 10261, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 5076, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 7370, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 11956, null, null, null, null, 1239, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1157, null, null, null, 2938, null, null, null, null, null, null, 14152, null, null, null, 4783, null, null, 1895, null, null, 2170, 6915, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null)",2017
730,"List(Multi-player, Valve Anti-Cheat enabled, Stats, Steam Achievements, Full controller support, Steam Trading Cards, Steam Workshop, In-App Purchases, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV)",874053,"Valve, Hidden Path Entertainment",0,"Action, Free to Play",0,"English, Czech, Danish, Dutch, Finnish, French, German, Hungarian, Italian, Japanese, Korean, Norwegian, Polish, Portuguese - Portugal, Portuguese - Brazil, Romanian, Russian, Simplified Chinese, Spanish - Spain, Swedish, Thai, Traditional Chinese, Turkish, Bulgarian, Ukrainian, Greek, Spanish - Latin America, Vietnamese",Counter-Strike: Global Offensive,787093,"50,0

Databricks visualization. Run in Databricks to view.

Results are interesting because some games are both part of Top 15 best/worst rated games, but it is also normal as some of these games are known for having lots of cheats and toxic behaviors from players.

##### Are there years with more releases ?

In [0]:
check_values("release_year")

+------------+
|release_year|
+------------+
|        2022|
|        2021|
|        2020|
+------------+
only showing top 3 rows

+------------+
|release_year|
+------------+
|        null|
|        1997|
|        1998|
+------------+
only showing top 3 rows



In [0]:
# Top 5 years with the most publishments
df_date = df \
    .groupBy("release_year") \
    .count() \
    .sort('count', ascending=False) \
    .limit(5)

display(df_date)

release_year,count
2021,8805
2020,8287
2018,7663
2022,7451
2019,6949


Databricks visualization. Run in Databricks to view.

It seems that covid may have had an impact on VG publishments, as 2020 and 2021 are the years where we have seen the most publishments.

##### How are the prizes distributed? Are there many games with a discount?

In [0]:
check_values("price")

+-----+
|price|
+-----+
|99900|
|29990|
|26999|
+-----+
only showing top 3 rows

+-----+
|price|
+-----+
|    0|
|   28|
|   29|
+-----+
only showing top 3 rows



In [0]:
# Distribution of prices
df_prices = df \
    .groupby("price") \
    .count()

display(df_prices)

price,count
463,1
3749,1
9900,1
2999,865
1025,1
1990,2
623,1
1699,185
31,6
1339,1


Databricks visualization. Run in Databricks to view.

In [0]:
check_values('discount')

+--------+
|discount|
+--------+
|      90|
|      89|
|      88|
+--------+
only showing top 3 rows

+--------+
|discount|
+--------+
|       0|
|      10|
|      12|
+--------+
only showing top 3 rows



In [0]:
# Proportion of discounted games
df_discount = df \
    .withColumn("discount", F.when(df["discount"] == 0, "No discount").otherwise("Discount")) \
    .groupBy("discount") \
    .count()

display(df_discount)

discount,count
No discount,53172
Discount,2518


Databricks visualization. Run in Databricks to view.

##### What are the most represented languages?

In [0]:
# Top 5 most represented languages in Steam
df_languages = df \
    .withColumn("languages", F.split(F.col("languages"), ",").cast("array<string>")) \
    .withColumn('languages', F.explode('languages')) \
    .withColumn('languages', F.regexp_replace('languages', 'Spanish - Spain', 'Spanish')) \
    .groupBy('languages') \
    .count() \
    .sort("count", ascending =False) \
    .limit(5)

display(df_languages)

languages,count
English,54646
German,13996
French,13406
Russian,12839
Spanish,12224


Databricks visualization. Run in Databricks to view.

##### Are there many games prohibited for children under16/18?

In [0]:
check_values('required_age')

+------------+
|required_age|
+------------+
|         180|
|          35|
|          20|
+------------+
only showing top 3 rows

+------------+
|required_age|
+------------+
|        null|
|           0|
|           3|
+------------+
only showing top 3 rows



We will consider "180" is a typo mistake for "18"

In [0]:
# Proportion of games forbidden to under-16s
df_age = df \
    .na.drop(subset=["required_age"]) \
    .withColumn("required_age", F.when(F.col("required_age") == 180, 18).otherwise(F.col("required_age"))) \
    .withColumn("age_above_16", F.when(F.col("required_age") >= 16, "Prohibited").otherwise("Not prohibited")) \
    .groupBy("age_above_16") \
    .count() \
    .sort("age_above_16", ascending=False)

display(df_age)

age_above_16,count
Prohibited,305
Not prohibited,55382


Databricks visualization. Run in Databricks to view.

Most of the games are authorized for children under 16.

## Data analysis - focus genres

In [0]:
# Prepare a dataset for the analysis
df_genres = df \
    .withColumn("genre", F.split(F.col("genre"), ",").cast("array<string>")) \
    .withColumn('genre', F.explode('genre')) \
    .withColumn("genre", F.trim(F.col("genre")))

##### What are the most represented genres?

In [0]:
# Top5 most represented genres
df_popularity = df_genres \
    .groupBy("genre") \
    .count() \
    .sort('count', ascending=False) \
    .limit(5)

display(df_popularity)

genre,count
Indie,39681
Action,23759
Casual,22086
Adventure,21431
Strategy,10895


Databricks visualization. Run in Databricks to view.

##### Are there any genres that have a better positive/negative review ratio ?

In [0]:
# Top 5 genres with better positive/negative ratios
df_ratio = df_genres \
    .groupBy('genre') \
    .agg(F.sum("positive").alias("total_positive"), F.sum("negative").alias("total_negative")) \
    .withColumn("positive_negative_ratio", F.col("total_positive") / F.col("total_negative")) \
    .sort("positive_negative_ratio", ascending=False) \
    .limit(5)

display(df_ratio)

genre,total_positive,total_negative,positive_negative_ratio
Photo Editing,577751,13745,42.03353946889778
Animation & Modeling,690765,26392,26.17327220369809
Design & Illustration,674057,27007,24.958603325063873
Utilities,739335,43503,16.99503482518447
Game Development,27461,3274,8.38759926695174


Databricks visualization. Run in Databricks to view.

##### Do some publishers have favorite genres?

In [0]:
# Top 15 publishers' favorite genres
df_favorite = df_genres \
    .groupBy("publisher", "genre") \
    .count() \
    .orderBy("count", ascending=False) \
    .groupBy("publisher") \
    .agg(F.max(F.struct("count", "genre")).alias("max_count_genre")) \
    .select("publisher", "max_count_genre.genre", "max_count_genre.count")

df_top15_favorites = df_dev.join(df_favorite, ['publisher'], 'inner')

display(df_top15_favorites)

publisher,count,genre,count
8floor,202,Casual,202
Alawar Entertainment,107,Casual,105
Big Fish Games,422,Casual,418
Choice of Games,140,RPG,139
Fulqrum Publishing,104,Strategy,56
HH-Games,132,Casual,132
Laush Studio,126,Indie,124
Plug In Digital,101,Indie,79
SEGA,165,Action,80
Sekai Project,132,Casual,99


Databricks visualization. Run in Databricks to view.

##### What are the most lucrative genres?

As we only have a range for game owners, I will take only the maximum number of the range to estimate the top most lucrative games.

In [0]:
# Top 5 most lucratives genres
df_profit = df_genres \
    .withColumn("owners", F.substring_index(F.col("owners"), "..", -1)) \
    .withColumn("owners", F.regexp_replace("owners", ",", "")) \
    .withColumn('owners', F.col("owners").cast(IntegerType())) \
    .withColumn("profit", F.col("price") * F.col("owners")) \
    .groupBy("genre") \
    .agg(F.sum("profit").alias("profit")) \
    .orderBy('profit', ascending=False) \
    .limit(5)

display(df_profit)

genre,profit
Indie,1867194192704
Action,1561294043344
Adventure,1377621048448
Strategy,783724305472
RPG,743458813616


Databricks visualization. Run in Databricks to view.

## Data analysis - focus platform

In [0]:
# Prepare a dataset for the next analysis on platforms and genres
df_platform = df \
    .select('genre', 'platforms', 'publisher') \
    .withColumn("platforms_list", F.concat_ws(", ",
                                               F.when(df["platforms"]["linux"], "Linux"),
                                               F.when(df["platforms"]["mac"], "Mac"),
                                               F.when(df["platforms"]["windows"], "Windows")))
    
df_platform = df_platform \
    .withColumn("platforms_list", F.split(F.col("platforms_list"), ",").cast("array<string>")) \
    .withColumn('platforms_list', F.explode('platforms_list')) \
    .withColumn("platforms_list", F.trim(F.col("platforms_list")))

##### Are most games available on Windows/Mac/Linux instead ?

In [0]:
# Ranking of most popular platforms
df_top_platform = df_platform \
    .groupBy('platforms_list') \
    .count() \
    .sort("count", ascending =False) \
    .limit(5)

display(df_top_platform)

platforms_list,count
Windows,55675
Mac,12769
Linux,8457


Databricks visualization. Run in Databricks to view.

##### Do certain genres tend to be preferentially available on certain platforms?

In [0]:
# Most published genres on platforms
df_genre_platform = df_platform \
    .withColumn("genre", F.split(F.col("genre"), ",").cast("array<string>")) \
    .withColumn('genre', F.explode('genre')) \
    .withColumn("genre", F.trim(F.col("genre"))) \
    .groupBy("platforms_list", "genre") \
    .count() \
    .orderBy("count", ascending=False) \
    .groupBy("platforms_list") \
    .agg(F.max(F.struct("count", "genre")).alias("max_count_genre")) \
    .select("platforms_list", "max_count_genre.genre", "max_count_genre.count")

df_genre_platform = df_popularity.join(df_genre_platform, ['genre'], 'inner')

display(df_genre_platform)

genre,count,platforms_list,count
Indie,39681,Windows,39676
Indie,39681,Mac,9935
Indie,39681,Linux,6978


Databricks visualization. Run in Databricks to view.